# Introduction à knn
---
Le but de ce TP est de mieux appréhender l'apprentissage via une méthode type 'plus proches voisins'. 
--- 

## Conseils
- Pour chaque question, essayer d'écrire des fonctions plutôt que des scripts
- Eviter d'avoir deux fonctions qui portent le même nom (redéfinition de la fonction), préférer ajouter un paramètre de choix

---
# Construction d'un jeu de données simple
---
Dans ce TP, on n'utili pas un jeu de données existant, mais on en fabrique un, d'un niveau de complexité adéquat, et dont on connait en réalité parfaitement la loi.

## Ecrire une fonction ```separation```
- Entrée :  un couple (x, y) de flottants
- Sortie : 'A' si y<f(x)  et 'B' sinon, où $f(x) = \frac{x}{3}*\left(2+\cos\left(\frac{x}{6}\right)\right)$

In [138]:
import math as m
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [139]:
def sep(x,y):
    fx = (x/3) * (2+m.cos(x/6))
    if(y < fx):
        return 'A'
    else :
        return 'B'

## Ecrire une fonction ```donneesSimples``` 
- Entrée : 
    - ```N``` un entier 
    - ```f``` une fonction prenant en entrée deux flottants
- Sortie : un dataframe de ```N``` échantillons de 3 attributs :
  - ```abscisse``` : tiré en aléatoire uniforme dans [0 ; 100], flottant
  - ```ordonnee``` : tiré en aléatoire uniforme dans [0 ; 100], flottant
  - ```classe = f(abscisse, ordonnée)```
- vous *devez* utiliser un apply de la fonction ```séparation```

In [140]:

def donneesSimples(N, f):
    d = {"abscisse":np.random.uniform(0,100,N),"ordonnée":np.random.uniform(0,100,N)}
    df = pd.DataFrame(data = d)
    df['classe'] = df.apply(lambda row: f(row['abscisse'], row['ordonnée']), axis = 'columns')
    return df

  
## Ecrire une fonction ```representation```
Cette fonction génère une représentation graphique des données dans le plan

In [141]:
def get_y(x):
    return (x/3) * (2+m.cos(x/6))
def representation(df):
    sns.scatterplot(x=df['abscisse'], y=df['ordonnée'], hue=df['classe'])
    
    df2 = df['abscisse']
    df2 = pd.DataFrame(df2)
    
    df2['ordonnée']= df2.apply(lambda row: get_y(row['abscisse']), axis = 'columns')
    sns.lineplot(x=df2['abscisse'], y=df2['ordonnée'])


## Créer une BD ```donnees``` de 600 données
- par application de ```donneesSimples(600 separation)```

In [142]:
df = donneesSimples(600, sep)

## Représenter la base de donnée graphiquement
Un scatterplot pour les points et un lineplot pour la séparation théorique donnent un graphique pertinent.

In [143]:
representation(df)

## En utilisant scikit-learn, séparer ```donnees``` en deux ensembles
Ecrire la fonction , fonction ```creerBases``` qui sépare la base de données en :
- apprentissage (et validation) : 50%
- validation et test 50%  
 
**NB** : dans ce TP, l'ensemble de test servira d'ensemble de validation.

**NB2** : à la fin de cette étape, on doit disposer de deux dataframes, un pour l'apprentissage, un pour validation/test

In [144]:
from sklearn.model_selection import train_test_split

def creerBases(df):

    X = df.iloc[:,:2]
    y = df.classe

    train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.5, stratify=y, random_state=1)
    
    return X,y,train_X, test_X,train_Y, test_Y 

X,y,train_X, test_X,train_Y, test_Y = creerBases(df)

# Théorie et sa mise en pratique dans sklearn
- Rappeler le fonctionnement de la classification par knn
- Expliquer les variations proposées par 'algorithm'
- Expliquer l'influence de 'p' dans la distance de Minkowski (exemples à l'appui)
- Expliquer l'influence de 'weight' (exemples à l'appui)

Le but de la classification par KNN c'est de déterminer la classe de points inconnu par rapport à leur distance avec leurs plus proches voisins dont la classe est connu

In [145]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score

knn = KNeighborsClassifier()
knn.fit(train_X,train_Y)
prediction = knn.predict(test_X)
display(balanced_accuracy_score(test_Y, prediction))

knn = KNeighborsClassifier(p = 1)
knn.fit(train_X,train_Y)
prediction = knn.predict(test_X)
display(balanced_accuracy_score(test_Y, prediction))

# Représenter la surface de séparation en fonction du nombre de voisins
## Fonction de coloration
Ecrire une fonction ```colorationPlan``` qui produit une représentation [0 ; 100]x[0 ; 100] colorée en fonction de la classe attribuée par votre classifieur.
en fonction de la classe attribuée par votre classifieur à chacun de ses points.
- Entrées : 
    - ```classifieur``` une fonction de classification (pour vérifier votre code, prenez un classifieur de votre choix dans sklearn)
    - ```NbPts``` nombre de points à prendre dans chaque direction pour discrétiser [0 ; 100]x[0 ; 100]

In [146]:
import itertools as it
def colorationPlan(classifieur, NbPts):
        X = np.linspace(0,100,NbPts)
        combinations = it.product(X,X)
        df = pd.DataFrame(combinations, columns =['abscisse', 'ordonnée'])
        predictions = knn.predict(df)
        df['classe'] = predictions
        representation(df)
colorationPlan(knn, 200)

## Observer et analyser l'influence du choix de 'k' sur la classification obtenue
Pour différentes valeurs de ```k``` dans le classifieur, utiliser la fonction ```colorationPlan``` et comparer les résultats.

In [147]:
def knnf(k):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(train_X,train_Y)
    return knn

knn = knnf(1)
colorationPlan(knn, 200)
plt.figure()
knn = knnf(3)
colorationPlan(knn, 200)
plt.figure()
knn = knnf(5)
colorationPlan(knn, 200)
plt.figure()
knn = knnf(11)
colorationPlan(knn, 200)

## Commenter
- Essayer d'expliquer les résultats obtenus
- Entre deux résultats équivalents pour deux valeurs de 'k' différentes, lequel choisir ?

Les résultats sont très similaires néanmoins, les points au-dessus de la courbe ont tendance à avoir la même classe que ceux en-dessous car leurs voisins direct sont en-dessous de celle-ci ce qui donne une impression de débordements dans les prédictions des valeurs au-dessus de la courbe.


Il faudrait prendre le k le plus petit car cela allège les calculs de voisins

## Recherche des paramètres optimaux
- utiliser une gridsearch pour déterminer les meilleurs paramètres, expliquer la valeur de cv

In [148]:
from sklearn.model_selection import GridSearchCV

def opti(trainx,trainy):

    knn     = KNeighborsClassifier()
    k_range = list(range(1,21))

    param_grid  = dict(n_neighbors = k_range)
    grid        = GridSearchCV(knn, param_grid, cv = 10, scoring = 'accuracy')
    grid_search = grid.fit(trainx, trainy)
    
    return grid_search.best_params_['n_neighbors']
opt = opti(train_X,train_Y)

cv représente le nombre de cross-validation

# Représenter la matrice de confusion sur l'ensemble de validation

In [149]:
from sklearn.metrics import plot_confusion_matrix

knn = KNeighborsClassifier(n_neighbors = opt)
knn.fit(train_X,train_Y)
plot_confusion_matrix(knn, test_X, test_Y)

# Créer deux nouvelles bases d'apprentissage une de 100 données, une de 1000 données
- Observer l'influence de la quantité de données sur les valeurs optimales du paramètre 'k', expliquer
- Observer l'influence de la quantité de données sur la qualité du résultat obtenu, expliquer (faire attention à mesurer la qualité de l'apprentissage sur l'ensemble validation/test !)
- Observer l'influence de la quantité de données sur le temps de calcul, expliquer

*NB* : afin d'obtenir des résultats pertinents, il est nécessaire de relancer ce travil un certain nombre de fois afin d'en dégager une tendance statistique.

In [150]:
import time
dfC = donneesSimples(100, sep)
dfM = donneesSimples(1000, sep)

XC,yC,train_XC, test_XC,train_YC, test_YC = creerBases(dfC)
XM,yM,train_XM, test_XM,train_YM, test_YM = creerBases(dfM)

#Déterminisation de la valeur optimale k

debC = time.time() 
optC = opti(train_XC, train_YC)
finC = time.time()

debM = time.time() 
optM = opti(train_XM, train_YM)
finM = time.time()

for k in range(1,6):
    debC2 = time.time() 
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(train_XC,train_YC)
    prediction = knn.predict(test_XC)
    display("Acc pour C pour k = " + str(k) + " : " + str(balanced_accuracy_score(test_YC, prediction)))
    finC2 = time.time() 
    
    debM2 = time.time() 
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(train_XM,train_YM)
    prediction = knn.predict(test_XM)
    display("Acc pour M pour k = " + str(k) + " : " + str(balanced_accuracy_score(test_YM, prediction)))
    finM2 = time.time()

display("Valeur optimale pour 100 :" + str(optC))
display("Valeur optimale pour 1000 :" + str(optM))

display("Temps écoulé pour 100 :" + str(finC - debC))
display("Temps écoulé pour 100 (knn) :" + str(finC2 - debC2))

display("Temps écoulé pour 1000 :" + str(finM - debM))
display("Temps écoulé pour 1000 (knn) :" + str(finM2 - debM2))

### Influence sur k

C : 1,4,3,1,1,1,1,1,1,19,1,1,1,1,6,3,1,2,1,1,6,1,1,3,3,16,2,3,1,1<br>
M : 1,1,1,3,1,8,9,1,1,1,1,3,1,11,1,5,3,1,1,1,1,2,1,1,1,3,1,1,3,1<br>

Valeur moyenne:<br>
C : 2.93<br>
M : 2.33<br>

Avec un échantillon de 30 valeurs on obtient que le k moyen pour M est inférieur à celui de C.
Ceci peut s'expliquer par le fait que plus on a de points, moins la distance entre ceux-ci est grande, ainsi les voisins ont tendances à être plus "proche" que lorsque le nombre de données est faible.

### Influence sur la précision

Résultats de 3 runs avec 5 précisions pour k allant de 1 à 5:<br>
C : 0.882, 0.878, 0.925, 0.878, 0.894, 0.778, 0.881, 0.823, 0.881, 0.881, 0.881, 0.849, 0.925, 0.865, 0.910 <br>
M : 0.980, 0.976, 0.956, 0.967, 0.961, 0.971, 0.976, 0.969, 0.962, 0.952, 0.972, 0.971, 0.968, 0.973, 0.953 <br>

Valeur moyenne: <br>
C : 0.875 <br>
M : 0.967 <br>

On remarque une différence de quasiment 10% de précision en plus pour la base contenant 1000 données, ce qui est énorme et montre que plus on a de données, mieux c'est.

### Influence sur le temps
C : 1.165, 1.081, 1.097, 1.116, 0.956 <br>
M : 1.707, 1.764, 1.714, 1.728, 1.351<br>

Valeur moyenne:<br>
C : 1.083<br>
M : 1.653<br>

Malgré le faible échantillon d'essai on se rend bien compte de manière assez logique que le nombre de données influe sur le temps de résolution de la recherche du k optimal. Plus on a de données, plus il faudra attendre pour résoudre ce problème

On tire la même conclusion pour le temps de calcul des knn

---
# Effet du bruit
Dans cette partie on va examiner la résistance au bruit de l'algorithme, et son influence sur le choix du nombre de voisins.
---
Cette étude a pour vocation de mieux comprendre le choix des paramètres lorsque certaines données sont incorrectes, ou lorsque le modèle contient des irrégularité non aisément modélisables (par exemple si la classe n'est pas totalement une fonction des entrées).

## Génération des données
Ecrire une fonction prenant en entrée un DataFrame 'T' servant à l'apprentissage et une probabilité $0\leq p\leq 1$ qui renvoie un nouveau dataframe qui est une copie de 'T', mais où la classe des exemples a été altérée avec la probabilité $p$ : pour chaque ligne du dataframe, la classe est inchangée avec la probabilité $1-p$, et modifiée avec la probabilité $p$.


In [151]:
def bruiteur(T,p):
    noised = T.copy()
    for i, row in T.iterrows():
        mod = np.random.random()
        if(mod <= p):
            if(row['classe'] == 'A'):
                noised.loc[i,'classe'] = 'B'
            if(row['classe'] == 'B'):
                noised.loc[i,'classe'] = 'A'
    return noised

p = np.random.random()

T = pd.concat([train_X,train_Y], axis = 1)

noised = bruiteur(T,p)

## Analyser l'effet de la force du bruit sur les meilleurs paramètres du modèle
Représenter le nombre optimal de voisins à choisir en fonction de la valeur du paramètre $p$.

In [152]:
opti_bruit = []
acc_bruit  = []
time_bruit = []

for i in np.arange(0, 100, 1):
    
    start = time.time()
    noised = bruiteur(T,i/100)
    XN, yN,train_XN, test_XN,train_YN, test_YN = creerBases(noised)
    tmp = opti(train_XN,train_YN)
    opti_bruit.append(tmp)
    
    #On prend pour le meilleur nombres de voisins pour chaque modèle
    knn = KNeighborsClassifier(n_neighbors = tmp)
    knn.fit(train_XN,train_YN)
    prediction = knn.predict(test_XN)
    acc_bruit.append(balanced_accuracy_score(test_YN, prediction))
    end = time.time()
    time_bruit.append(end-start)
    
sns.lineplot(data = opti_bruit)
plt.figure()
sns.lineplot(data = acc_bruit)
plt.figure()
sns.lineplot(data = time_bruit)

## Conclure, et vérifier l'effet sur les autres paramètres

### Influence du bruit sur le nombre optimal de k

Comme nous pouvons le voir dans le premier graphique, avec un bruit <20% (réciproquement >80%) le nombre de voisins excède rarement les 12. Néanmoins à partir d'une probabilité de 20% de bruit (80% en réciproque) les nombre de voisins semble être totalement aléatoire.

### Influence du bruit sur la précision

Plus le bruit approche les 50% plus la précision fait de même, elle s'approche de l'aléatoire. Ainsi le knn est aussi précis qu'un inconnu qui affirmerait haut et fort l'appartenance à une classe d'une ligne pour chacune.

### Influence du bruit sur le temps

On observe une croissance du temps de calcul, néanmoins elle est sûrement dû au fait que l'aggrégation se fait de plus en plus probable, donc il y en a plus et cela coûte du temps.

### Conclusion
Le bruit dans notre cas n'affecte que 2 classes, il serait intéressant d'y faire sur plusieurs afin d'avoir une vraie conclusion sur celui-ci.
Dans notre cas, il s'agit d'une inversion de classe donc les résultats obtenus sont quasi simétriques.

---
# Influence des corrélations entre les attributs
---
On travaille à nouveau sur une base de données non bruitée. Ecrire une fonction prenant en entrée un DataFrame de même nature que train ou test, et qui en renvoie une copie à laquelle on a ajouté 5 colonnes définies par :
- U1 :   $+300 \times X + 0.1 \times Y$
- U2 :   $-4000 \times X - 0.01 \times Y$
- U3 :   $-700 \times X + 0.12 \times Y$
- U4 :   $-280 \times X + 1.5 \times Y$
- U3 :   $+5100 \times X - 1.2 \times Y$  

NB : Remettre la colonne ```classe``` en dernière colonne.

In [153]:
dfA = donneesSimples(500, sep)
XA, yA,train_XA, test_XA,train_YA, test_YA = creerBases(dfA)
dfAT = pd.concat([train_XA, train_YA], axis = 1)
dfAP = pd.concat([test_XA, test_YA], axis = 1)

def correlation(df):
    idx = []
    u1 = []
    u2 = []
    u3 = []
    u4 = []
    u5 = []
    classe = df['classe']
    df.drop(['classe'], axis=1)
    for i, row in df.iterrows():
        
        # Création des données
        idx.append(i)
        u1.append(300*row['abscisse'] + 0.1*row['ordonnée'])
        u2.append(-4000*row['abscisse'] - 0.01*row['ordonnée'])
        u3.append(-700*row['abscisse'] + 0.12*row['ordonnée'])
        u4.append(-280*row['abscisse'] + 1.5*row['ordonnée'])
        u5.append(5100*row['abscisse'] - .2*row['ordonnée'])
    
    # Conversion en Series pour concat
    u1 = pd.Series(u1,index=idx)
    u2 = pd.Series(u2,index=idx)
    u3 = pd.Series(u3,index=idx)
    u4 = pd.Series(u4,index=idx)
    u5 = pd.Series(u5,index=idx)  
    df = pd.concat([df,u1,u2,u3,u4,u5,classe], axis=1)
    
    return df
dfCor = correlation(dfAT)
dfCor2 = correlation(dfAP)
display(dfCor.head())

# Observer la diférence de qualité entre :
- Apprendre en utilisant toutes les observations (x, y, u1, u2, u3, u4, u5)
- Apprendre en n'utilisant que les observations (x,y)

In [154]:
train_X7 = dfCor.copy().drop(['classe'],axis=1)
test_X7 = dfCor2.copy().drop(['classe'],axis=1)

# Test avec (x,y,u1,u2,u3,u4,u5) pour X
opt = opti(train_XA,train_YA)

knn = KNeighborsClassifier(n_neighbors = opt)
knn.fit(train_X7,train_YA)
prediction = knn.predict(test_X7)
display(balanced_accuracy_score(test_YA, prediction)) #Le Y de test ne change pas 


# Test avec (x,y) pour X
opt = opti(train_XA,train_YA)

knn = KNeighborsClassifier(n_neighbors = opt)
knn.fit(train_XA,train_YA)
prediction = knn.predict(test_XA)
display(balanced_accuracy_score(test_YA, prediction))


## Expliquer la raison de la différence observée

Les valeurs de u1,u2,u3,u4,u5 affectent les distances autant que x et y mais sont très axès sur la valeur de X ce qui dérègle l'importance des valeurs d'abscisse et d'ordonnée.<br>
Au mieux cela n'aurait aucun effet, au pire comme c'est le cas ici présent, cela réduit notre probabilité de bonne prédiction

## Utiliser un preprocessing adapté sur les données afin que la qualité obtenue en apprenant sur le jeu de données (x, y, u1, u2, u3, u4, u5) soit comparable à celle obtenue en n'utilisant que (x, y)

expliquer et pointer les limitations, proposer des améliorations


---
# Merci d'avoir suivi ce TP, j'espère qu'il vous a aidé à mieux appréhender l'utilisation de knn, et vous a permis de faire quelques pas dans le domaine *passionnant* de l'apprentissage artificiel